# Fictitious Names

### Introduction:

This time you will create a data again 

Special thanks to [Chris Albon](http://chrisalbon.com/) for sharing the dataset and materials.
All the credits to this exercise belongs to him.  

In order to understand about it go [here](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/).

### Step 1. Import the necessary libraries

In [70]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T


# Generando session

spark = SparkSession.builder\
                     .appName('fakeNames')\
                     .getOrCreate()
                     
                     

### Step 2. Create the 3 DataFrames based on the following raw data

In [71]:
raw_data_1 = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}

raw_data_2 = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}

raw_data_3 = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}

### Step 3. Assign each to a variable called data1, data2, data3

In [72]:
df1 = pd.DataFrame(raw_data_1)
df2 = pd.DataFrame(raw_data_2)
df3 = pd.DataFrame(raw_data_3)

In [73]:
data1 = spark.createDataFrame(df1)
data2 = spark.createDataFrame(df2)
data3 = spark.createDataFrame(df3)


In [74]:
data = []
for i, _ in enumerate(raw_data_1.items()):
    
    combinado = [raw_data_1[key][i] for key in raw_data_1.keys()]
    
    data.append(tuple(combinado))


data

[('1', 'Alex', 'Anderson'), ('2', 'Amy', 'Ackerman'), ('3', 'Allen', 'Ali')]

In [75]:
schema = T.StructType([
    T.StructField('subject_id', T.StringType()),
    T.StructField('first_name', T.StringType()),
    T.StructField('last_name', T.StringType()),

])

In [76]:
otra_forma = spark.createDataFrame(data, schema=schema)

In [77]:
otra_forma.show()

+----------+----------+---------+
|subject_id|first_name|last_name|
+----------+----------+---------+
|         1|      Alex| Anderson|
|         2|       Amy| Ackerman|
|         3|     Allen|      Ali|
+----------+----------+---------+



### Step 4. Join the two dataframes along rows and assign all_data

In [78]:
all_data = data1.union(data2)


In [79]:
all_data.show()

+----------+----------+---------+
|subject_id|first_name|last_name|
+----------+----------+---------+
|         1|      Alex| Anderson|
|         2|       Amy| Ackerman|
|         3|     Allen|      Ali|
|         4|     Alice|     Aoni|
|         5|    Ayoung|  Atiches|
|         4|     Billy|   Bonder|
|         5|     Brian|    Black|
|         6|      Bran|  Balwner|
|         7|     Bryce|    Brice|
|         8|     Betty|   Btisan|
+----------+----------+---------+



In [80]:
df_all = pd.concat([df1, df2], axis=0)
df_all

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


### Step 5. Join the two dataframes along columns and assing to all_data_col

In [81]:
all_data.join(data2, on='subject_id').show()


+----------+----------+---------+----------+---------+
|subject_id|first_name|last_name|first_name|last_name|
+----------+----------+---------+----------+---------+
|         4|     Alice|     Aoni|     Billy|   Bonder|
|         4|     Billy|   Bonder|     Billy|   Bonder|
|         5|    Ayoung|  Atiches|     Brian|    Black|
|         5|     Brian|    Black|     Brian|    Black|
|         6|      Bran|  Balwner|      Bran|  Balwner|
|         7|     Bryce|    Brice|     Bryce|    Brice|
|         8|     Betty|   Btisan|     Betty|   Btisan|
+----------+----------+---------+----------+---------+



In [82]:

df_12= pd.concat([df1, df2], axis=1)

df_12

,subject_id,first_name,last_name,subject_id,first_name,last_name
0,1,Alex,Anderson,4,Billy,Bonder
1,2,Amy,Ackerman,5,Brian,Black
2,3,Allen,Ali,6,Bran,Balwner
3,4,Alice,Aoni,7,Bryce,Brice
4,5,Ayoung,Atiches,8,Betty,Btisan


### Step 6. Print data3

In [83]:
data3.show()

+----------+-------+
|subject_id|test_id|
+----------+-------+
|         1|     51|
|         2|     15|
|         3|     15|
|         4|     61|
|         5|     16|
|         7|     14|
|         8|     15|
|         9|      1|
|        10|     61|
|        11|     16|
+----------+-------+



In [84]:
df3.head()

,subject_id,test_id
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16


### Step 7. Merge all_data and data3 along the subject_id value

In [85]:
all_data.join(data3, on='subject_id').show()

+----------+----------+---------+-------+
|subject_id|first_name|last_name|test_id|
+----------+----------+---------+-------+
|         1|      Alex| Anderson|     51|
|         2|       Amy| Ackerman|     15|
|         3|     Allen|      Ali|     15|
|         4|     Alice|     Aoni|     61|
|         4|     Billy|   Bonder|     61|
|         5|    Ayoung|  Atiches|     16|
|         5|     Brian|    Black|     16|
|         7|     Bryce|    Brice|     14|
|         8|     Betty|   Btisan|     15|
+----------+----------+---------+-------+



In [88]:
df_all.merge(df3, on='subject_id')

,subject_id,first_name,last_name,test_id
0,1,Alex,Anderson,51
1,2,Amy,Ackerman,15
2,3,Allen,Ali,15
3,4,Alice,Aoni,61
4,5,Ayoung,Atiches,16
5,4,Billy,Bonder,61
6,5,Brian,Black,16
7,7,Bryce,Brice,14
8,8,Betty,Btisan,15


### Step 8. Merge only the data that has the same 'subject_id' on both data1 and data2

### Step 9. Merge all values in data1 and data2, with matching records from both sides where available.